## MovieLens 100K Datensatz mit Features

Entnommen wurde der Datensatz aus https://grouplens.org/datasets/movielens/100k/. Dieser ist der **10-Core**
MovieLens Datensatz mit 100.000 Bewertungen.

**Datensatz-Quelle:** <br />
F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets:<br />
History and Context. ACM Transactions on Interactive Intelligent<br />
Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages.<br />
DOI=http://dx.doi.org/10.1145/2827872

Die Implementation des Mini-Batche-Samplings (<i>class MinibatchSampler</i>) und des GCMC-Modells wurde (<i>class GCMCConv, class GCMCLayer, GCMCRating </i>) von der DGL-Library für das MovieLens Datenset zu Verfügung gestellt.

**DGL:**<br />
Minjie Wang and Da Zheng and Zihao Ye and Quan Gan and Mufei Li and Xiang Song and Jinjing Zhou and Chao Ma and Lingfan Yu and Yu Gai and Tianjun Xiao and Tong He and George Karypis and Jinyang Li and Zheng Zhang (2019): <br />
Deep Graph Library: A Graph-Centric, Highly-Performant Package for Graph Neural Networks <br />
arXiv preprint arXiv = 1909.01315


**Implementation eines hybriden Recommender Systems:** <br />
Bei diesem Informationssystem handelt es sich um ein hybrides Recommender System, welches das 
**Collaborative, Content-based und Demographic Filtering** nutzt. 
Überwiegend wird das Collaborative Filtering angewendet. Es darüber hinaus Inhalte von Filmen als Itemfeatures
und demographische Daten der User als Userfeatures genutzt werden. 


In [1]:
# Import des Datensatzes und der nötigen Bibliotheken
import pandas as pd

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F

import dgl
import dgl.function as fn
import dgl.nn as dglnn

from timeit import default_timer as timer
from datetime import timedelta
import math

# Datensatzvorbereitung
users = pd.read_csv('u.user', sep='|', header = None, encoding='latin-1',
                    names=['userId','alter','geschlecht', 'beruf','PLZ']) # mit | trennen damit Spalten generiert werden

movies = pd.read_csv('u.item', sep='|', header = None,encoding='latin-1', 
                     names=['itemId','title','veröffentlichung', 'NaN',
                            'links', 'unknown','action', 'adventure', 'animation',
                            'childrens','comedy', 'crime', 'documentary',
                            'drama', 'fantasy', 'filmnoir', 'horror', 'musical',
                            'mystery', 'romance', 'scifi', 'thriller', 'war',
                            'western']) 
genre = pd.read_csv('u.genre', sep='|', header = None,encoding='latin-1',
                    names=['genre', 'genreZahl']) # Werte für die Genres

trainset = pd.read_csv('ua.base', sep='\t', header = None,
                       names=['userId', 'itemId', 'rating', 'timestamp']) # mit \t trennen damit Spalten generiert werden

testset = pd.read_csv('ua.test', sep='\t', header = None, 
                      names=['userId', 'itemId', 'rating', 'timestamp'])

Using backend: pytorch


In [2]:
# Entfernung von unbrauchbaren Features
del trainset['timestamp']
del testset['timestamp']
del users['PLZ']
del movies['title']
del movies['veröffentlichung']
del movies['NaN']
del movies['links']


In [3]:
# entferne Items von testset die nicht im trainset sind. 
ueberschussTests = list(set(testset['itemId']).difference(set(trainset['itemId']))) #speichere differenz der Menge
i = 0
while i< len(ueberschussTests):  
    index = list(testset['itemId']).index(ueberschussTests[i]) #speichere Index vom Ueberschuss
    testset = testset.drop([index]) #lösche Items die überschüssig sind. 
    i = i+1

# entferne Items von movies die nicht im trainset sind.
ueberschussMovies = list(set(movies['itemId']).difference(set(trainset['itemId']))) #speichere Differenz der Menge
i = 0
while i< len(ueberschussMovies):  
    index = list(movies['itemId']).index(ueberschussMovies[i])
    movies = movies.drop([index]) # lösche Items die überschüssig sind. 
    i = i+1

### Konstruiere bipartiten Graphen

In [4]:
# Erstellt einen Tensor aus einer Liste. Ordnet Werte der Liste
# zu Unique-Values zu, falls bool = True. Sonst nicht.
def buildTensor(list, bool): 

    if bool:
        list = torch.LongTensor(list.astype('category')
                                .cat.codes.values) # Konvertiere zu category damit cat.codes ausgeführt werden kann.
    else:
        list = torch.LongTensor(list.values)
    return list

In [5]:
# Bilde Tensoren User und Item für den Graphen
# Zuordnung der Ids, da diese bei 1 Anfangen und nicht bei 0.
userId = buildTensor(trainset['userId'], True)
itemId = buildTensor(trainset['itemId'], True)

# Bilde Tensoren User und Item zum Testen des Graphens
userIdTest = buildTensor(testset['userId'], True)
itemIdTest = buildTensor(testset['itemId'], True)


# Erstelle bipartiten Graphen
graph = dgl.heterograph({
    ('user', 'rated', 'item'): (userId, itemId),
    # In DGL exestieren nur gerichtete Graphen. Daher wird
    # dies über beide Richtungen definiert
    ('item', 'rated-by', 'user'): (itemId, userId)
})
 
print(graph) # Info über Graphen

Graph(num_nodes={'item': 1680, 'user': 943},
      num_edges={('item', 'rated-by', 'user'): 90570, ('user', 'rated', 'item'): 90570},
      metagraph=[('item', 'user', 'rated-by'), ('user', 'item', 'rated')])


<ipython-input-4-218b375f64b8>:6: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  list = torch.LongTensor(list.astype('category')


### Features generieren

In [6]:
# User-Features über die demographische Daten der User generieren
userAlter = buildTensor(users['alter'] // 10, False) # Floordivision durch 10
userGeschlecht = buildTensor(users['geschlecht'], True)
userBeruf = buildTensor(users['beruf'], True)

# Item-Features über die Inhalte (Genre, Tags, Kategorien, etc.) generieren
moviesTypen = movies[['unknown','action', 'adventure', 'animation',
        'childrens','comedy', 'crime', 'documentary',
        'drama', 'fantasy', 'filmnoir', 'horror', 
        'musical', 'mystery', 'romance', 'scifi', 
        'thriller', 'war', 'western']].values

# Anzahl der Unique Values der Features
userAlterNum = len(set(users['alter']//10))
userGeschlechtNum = len(set(users['geschlecht']))
userBerufNum = len(set(users['beruf']))
moviesTypenNum = len(genre['genre'])

### Features zu den Knoten und Ratings zu den Kanten hinzufügen

In [7]:
# Füge User-Features zu den User-Knoten hinzu
graph.nodes['user'].data['alter'] = userAlter
graph.nodes['user'].data['geschlecht'] = userGeschlecht
graph.nodes['user'].data['beruf'] = userBeruf

# Füge Item-Features zu den Item Knoten hinzu
graph.nodes['item'].data['filmTyp'] = torch.FloatTensor(moviesTypen)

# Füge die Ratings zu den Kanten hinzu
bewertungen = buildTensor(trainset['rating'], False)
bewertungenTest = buildTensor(testset['rating'], False) # Für spätere Testzwecke
graph.edges['rated'].data['rating'] = bewertungen
graph.edges['rated-by'].data['rating'] = bewertungen

### Definiere Train und Testsets als Tensor für das Modell

In [8]:
# Definiere Test und Trainset für das Modell über Tensoren
tensorTrainset = TensorDataset(userId, itemId, bewertungen)
tensorTestset = TensorDataset(userIdTest, itemIdTest, bewertungenTest)

### Erstellung von Minibatches & Neighbor Sampling

In [9]:
# Ausführung des Mini-Batchings
class MinibatchSampler(object):

    def __init__(self, graph, num_layers):
        self.graph = graph
        self.num_layers = num_layers
        
    # Finde die nötigen Knoten und konstruiere den Pair Graphen
    def sample(self, batch):
        # Konvertiere die Liste des Batches (Trainset) in 3 verschiedenen Vektoren.
        users, items, ratings = zip(*batch)
        users = torch.stack(users)
        items = torch.stack(items)
        ratings = torch.stack(ratings)
        
        # Konstruiere Bipartiten-Graph auf Grundlage des Batches.
        pair_graph = dgl.heterograph(
            {('user', 'rated', 'item'): (users, items)},
            num_nodes_dict={'user': self.graph.number_of_nodes('user'), 'item': self.graph.number_of_nodes('item')})
        

        pair_graph = dgl.compact_graphs(pair_graph) # Löscht Knoten die für das Sampling nicht genutzt werden.
        pair_graph.edata['rating'] = ratings # Füge die Ratings in die Sampling-Edges hinzu.
        
        # Konstruiere Blocks
        seeds = {'user': pair_graph.nodes['user'].data[dgl.NID], # Bestimme Seed-nodes über NID (Note-ID)
                 'item': pair_graph.nodes['item'].data[dgl.NID]}
        blocks = self.construct_blocks(seeds, (users, items)) # Hier werden Blöcke für die Seed-nodes konstruiert.
        
        # Speichert Node-Features vom Input Graphen in die Samplings.
        for feature_name in self.graph.nodes['user'].data.keys():
            blocks[0].srcnodes['user'].data[feature_name] = \
                self.graph.nodes['user'].data[feature_name][blocks[0].srcnodes['user'].data[dgl.NID]] 
            
        for feature_name in self.graph.nodes['item'].data.keys():
            blocks[0].srcnodes['item'].data[feature_name] = \
                self.graph.nodes['item'].data[feature_name][blocks[0].srcnodes['item'].data[dgl.NID]]
        
        # Ergebnis ist ein Mini-Batch
        return pair_graph, blocks 
    
    # Berechne den Block
    def construct_blocks(self, seeds, user_item_pairs_to_remove):
        blocks = []
        users, items = user_item_pairs_to_remove
        for i in range(self.num_layers):
            
            # übernimmt alle Nachbarn von den Seed-Nodes
            sampled_graph = dgl.in_subgraph(self.graph, seeds) 
            # Sampling für beide Richtungen finden.
            sampled_eids = sampled_graph.edges['rated'].data[dgl.EID]
            sampled_eids_rev = sampled_graph.edges['rated-by'].data[dgl.EID]
            
            # Hier werden die Kanten entfernt die nicht im Training-Prozess benötigt werden. 
            _, _, edges_to_remove = sampled_graph.edge_ids(users, items, etype='rated', return_uv=True)  
            _, _, edges_to_remove_rev = sampled_graph.edge_ids(items, users, etype='rated-by', return_uv=True)
            
            sampled_with_edges_removed = sampled_graph
            if len(edges_to_remove) > 0:
                sampled_with_edges_removed = dgl.remove_edges(
                    sampled_with_edges_removed, edges_to_remove, 'rated')
                sampled_eids = sampled_eids[sampled_with_edges_removed.edges['rated'].data[dgl.EID]]
            if len(edges_to_remove_rev) > 0:
                sampled_with_edges_removed = dgl.remove_edges(
                    sampled_with_edges_removed, edges_to_remove_rev, 'rated-by')
                sampled_eids_rev = sampled_eids_rev[sampled_with_edges_removed.edges['rated-by'].data[dgl.EID]]
            
            # Konstruiere einen Block vom gesampelten Graphen
            block = dgl.to_block(sampled_with_edges_removed, seeds)
            blocks.insert(0, block)
            seeds = {'user': block.srcnodes['user'].data[dgl.NID],
                     'item': block.srcnodes['item'].data[dgl.NID]}
            
            # Kopiere die Bewertungen zu den Kanten der gesampelten Graphen
            block.edges['rated'].data['rating'] = \
                self.graph.edges['rated'].data['rating'][sampled_eids]
            block.edges['rated-by'].data['rating'] = \
                self.graph.edges['rated-by'].data['rating'][sampled_eids_rev]
            
        return blocks

### Definiere das GCMC-Modell


In [10]:
from torch import nn
import torch.nn.functional as F
import dgl.function as fn
import dgl.nn as dglnn

class GCMCConv(nn.Module): # Die Funktionen dieser Klasse beschreibt einen Encoder

    def __init__(self, hidden_dims, num_ratings):
        super().__init__()
        
        # Die Bewertungen sind von 1 bis 5 nummeriert, daher muss eine 1 addiert werden. 
        self.W_r = nn.Parameter(torch.randn(num_ratings + 1, hidden_dims, hidden_dims)) # Initialisierung der erlernbaren Gewichtsmatrix in der Messagefunktion
        self.W_i = nn.Linear(hidden_dims * 2, hidden_dims) # Die Gewichtsmatrix W_i in der Akkumulationsfunktion
        
    def compute_message(self, W, edges): 
        W_r = W[edges.data['rating']] # W_r als Edge Feature für das Rating r. 
        h = edges.src['h'] # h als die h-te Zwischenebene des Encoders im GCMC, sowohl für User und Item
        m = (W_r @ h.unsqueeze(-1)).squeeze(2) # m als die Berechnete Message μ
        return m

    def forward(self, graph, node_features):
        with graph.local_scope():
            src_features, dst_features = node_features
            
            # Anwendung von compute_message auf alle Edgefeatures des Inputs
            graph.srcdata['h'] = src_features 
            graph.dstdata['h'] = dst_features 
            
             # Die Aggregation, wobei dies über den Schnitt der Nachbarschaft erfolgt
            graph.apply_edges(lambda edges: {'m': self.compute_message(self.W_r, edges)})
            
            # Normalisierung und Update 
            graph.update_all(fn.copy_e('m', 'm'), fn.mean('m', 'h_neigh')) 
            
            # Akkumulationsfunktion  mit Konkatenation
            result = F.relu(self.W_i(torch.cat([graph.dstdata['h'], graph.dstdata['h_neigh']], 1))) 
            return result 

In [11]:
class GCMCLayer(nn.Module):

    def __init__(self, hidden_dims, num_ratings):
        super().__init__()
        
        # Hier werden die Ebenen h_ui und h_vj für beide Richtungen ausgerechnet.
        self.heteroconv = dglnn.HeteroGraphConv(
            {'rated': GCMCConv(hidden_dims, num_ratings), 'rated-by': GCMCConv(hidden_dims, num_ratings)},
            aggregate='sum')
                
    def forward(self, block, input_user_features, input_item_features):
        with block.local_scope():
            # Input-Vektoren für die h.te Ebene
            h_user = input_user_features 
            h_item = input_item_features
            
            # übernehme Features von vorherhiger Ebene für nächste Ebene
            src_features = {'user': h_user, 'item': h_item} 
            dst_features = {'user': h_user[:block.number_of_dst_nodes('user')],
                            # Analog, jedoch mit Beachtung der Samplings
                            'item': h_item[:block.number_of_dst_nodes('item')]} 
            
            result = self.heteroconv(block, (src_features, dst_features))
            return result['user'], result['item']

In [12]:
class GCMCRating(nn.Module):
    def __init__(self, num_users, num_items, hidden_dims, num_ratings, num_layers):
        super().__init__()
        
        # Embeddingvektor der Größe hidden_dims für User und Items
        self.user_embeddings = nn.Embedding(num_users, hidden_dims)  
        self.item_embeddings = nn.Embedding(num_items, hidden_dims)
        
        # Embeddingvektor der Größe hidden_dims für Item-Features und User-Features
        self.U_age = nn.Embedding(userAlterNum, hidden_dims)
        self.U_gender = nn.Embedding(userGeschlechtNum, hidden_dims)
        self.U_occupation = nn.Embedding(userBerufNum, hidden_dims)
        self.U_genres = nn.Linear(moviesTypenNum, hidden_dims)
        
        self.layers = nn.ModuleList([
            # Berechnung der Layers im Encoder.
            GCMCLayer(hidden_dims, num_ratings) for _ in range(num_layers)]) 
        
        # Trainierbarer Parameter für Items W_v und für Users W_u
        self.W_u = nn.Linear(hidden_dims, hidden_dims) # Linear() besteht aus x*W_u^T + b wobei b ein Bias ist.
        self.W_v = nn.Linear(hidden_dims, hidden_dims)
        
    def forward(self, blocks):
        # Beginne mit Embedding für jeden User und Item
        user_embeddings = self.user_embeddings(blocks[0].srcnodes['user'].data[dgl.NID])
        item_embeddings = self.item_embeddings(blocks[0].srcnodes['item'].data[dgl.NID])
        
        # Addiere die Item und User Features zu den jeweiligen User und Items dazu. 
        user_embeddings = user_embeddings + self.U_age(blocks[0].srcnodes['user'].data['alter']) 
        user_embeddings = user_embeddings + self.U_gender(blocks[0].srcnodes['user'].data['geschlecht'])
        user_embeddings = user_embeddings + self.U_occupation(blocks[0].srcnodes['user'].data['beruf'])
        item_embeddings = item_embeddings + self.U_genres(blocks[0].srcnodes['item'].data['filmTyp'])
        
        # Iteriere über die Encoder-Layers
        for block, layer in zip(blocks, self.layers):
            # Berechnung der Nachricht zwischen Item und User
            user_embeddings, item_embeddings = layer(block, user_embeddings, item_embeddings) 
        
        # Zusammensetzung des Embeddingvektors und dazugehörigen trainierbaren Gewichtung 
        z_u = self.W_u(user_embeddings) 
        z_v = self.W_v(item_embeddings)
        
        return z_u, z_v # Finale Repräsentation der Knoten als Embeddingvektoren z_u und z_v
        
        # Decoder über das Skalarprodukt  
    def compute_score(self, pair_graph, z_u, z_v):
        with pair_graph.local_scope():
            # Nutze für die Ebene h die Embeddings z_u und z_v
            pair_graph.nodes['user'].data['h'] = z_u 
            pair_graph.nodes['item'].data['h'] = z_v
            
            # Berechne Rating über Skalarpodukt über z_u und z_v  und update die Kantenfeatures
            pair_graph.apply_edges(fn.u_dot_v('h', 'h', 'r')) 
            
            return pair_graph.edata['r'] #Ende des Forward-Propagation vom GCMC-Modell

### Training des GCMC-Modells

In [13]:
import tqdm
from sklearn.metrics import mean_squared_error

def trainingLoop(NUM_LAYERS, BATCH_SIZE, NUM_EPOCHS, HIDDEN_DIMS, NUM_RATINGS, printing = True):
    sampler = MinibatchSampler(graph, NUM_LAYERS) # Erstellt ein Sampler Objekt basierend auf den Graphen
    
    # Sampelt und erstellt Trainset und Testset auf Basis des Samplers und Batchsize
    train_dataloader = DataLoader(tensorTrainset, batch_size=BATCH_SIZE, collate_fn=sampler.sample, shuffle=True)
    test_dataloader = DataLoader(tensorTestset, batch_size=BATCH_SIZE, collate_fn=sampler.sample, shuffle=False)
        
    # Übergabe der Hyper-Parameter und Konstruktion des Modells für das Datenset
    model = GCMCRating(graph.number_of_nodes('user'), graph.number_of_nodes('item'), HIDDEN_DIMS, NUM_RATINGS, NUM_LAYERS) 
    
    # SGD-Optimierungsverfahren für die Modell-Parameter mit Lernparameter = 0.01
    opt = torch.optim.SGD(model.parameters(), lr=0.01) 
    
    rmse = []
    
    for i in range(NUM_EPOCHS):
        
        model.train() # Modell wird nun in Trainzustand gesetzt.
       
        with tqdm.tqdm(train_dataloader) as t: # Training über Trainset
            for pair_graph, blocks in t:
                user_emb, item_emb = model(blocks)
                prediction = model.compute_score(pair_graph, user_emb, item_emb)
                loss = ((prediction - pair_graph.edata['rating']) ** 2).mean()
                opt.zero_grad() # setze Gradienten auf 0
                loss.backward() # Berechne Gradienten mittels Backpropagation
                opt.step() # update die Modell-Parameter

        model.eval() # Modell wird nun in Testzustand gesetzt.
    
        with tqdm.tqdm(test_dataloader) as t: # Evaluation über Testset
            with torch.no_grad():
                predictions = []
                ratings = []
                for pair_graph, blocks in t:
                    # Definiere die  Embeddingvektoren von User und Item
                    user_emb, item_emb = model(blocks) 
                    # Berechnung der Vorhersage eines Ratings r
                    prediction = model.compute_score(pair_graph, user_emb, item_emb) 
                    predictions.append(prediction) # vorhergesagter wert von r
                    ratings.append(pair_graph.edata['rating']) # tatsächlicher Wert von r

                predictions = torch.cat(predictions, 0)
                ratings = torch.cat(ratings, 0)
        
        # Ausgabe des RMSE nach jedem SGD-Schritt
        if printing:
            print('RMSE:', mean_squared_error(predictions, ratings, squared=True).item() , ' - Nach',i+1,'. Epoch:')
        
        rmse.append(mean_squared_error(predictions, ratings, squared=True).item())
    
    # Gibt den endgültigen RMSE aus, falls printing = True
    if printing:
        print('\n\nAuswertung für folgende Hyper-Parameter: \n',
              'NUM_LAYERS','=', NUM_LAYERS, '\n',
              'BATCH_SIZE','=', BATCH_SIZE, '\n',
              'NUM_EPOCHS','=', NUM_EPOCHS, '\n',
              'HIDDEN_DIMS','=', HIDDEN_DIMS, '\n') 
        print('Endgültiger RMSE:', mean_squared_error(predictions, ratings, squared=True).item())
    
    return rmse

### Beispiel Hyper-Parameter für das Modell

In [ ]:
# Hyper-Parameter des GCMC-Modells
NUM_LAYERS = 1 # Ebenen des Encoders
BATCH_SIZE = 400 # Batch-Size für das Sampling
NUM_EPOCHS = 15 # Anzahl der SGD Iterationen
HIDDEN_DIMS = 4 # Länge des Vektors für einen Knoten
NUM_RATINGS = len(set(trainset['rating'])) # Anzahl der Bewertungselemente der Bewertungsmenge

rmse = trainingLoop(NUM_LAYERS, BATCH_SIZE, NUM_EPOCHS, HIDDEN_DIMS, NUM_RATINGS) 

  0%|▎                                                                                 | 1/227 [00:00<00:24,  9.17it/s]

RMSE: 1.301891166665733  - Nach 1 . Epoch:


  0%|▎                                                                                 | 1/227 [00:00<00:25,  9.01it/s]

RMSE: 1.2686960656386177  - Nach 2 . Epoch:


  0%|▎                                                                                 | 1/227 [00:00<00:24,  9.26it/s]

RMSE: 1.2682718582398786  - Nach 3 . Epoch:


  0%|▎                                                                                 | 1/227 [00:00<00:24,  9.09it/s]

RMSE: 1.2593827974124592  - Nach 4 . Epoch:


 39%|███████████████████████████████▊                                                 | 89/227 [00:09<00:16,  8.60it/s]

### Grid Search (für die Evaluation)

Für NUM_EPOCHS wird aufgrund des hohen Rechenaufwands der Wert auf 15 und die dazugehörige Lernrate auf 0.01 gesetzt.

In [14]:
layers = [1,2] # Ebenen des Encoders
batchsizes = [400, 600, 800] # Batch-Size für das Sampling
hiddendims = [4, 6, 8] # Länge des Vektors für einen Knoten
NUM_EPOCHS = 15
NUM_RATINGS = len(set(trainset['rating']))

evalDf = pd.DataFrame()

for layer in layers:
    for batchsize in batchsizes:
        for hiddendim in hiddendims:
            # Startzeit
            start = timer() 
            
            #definiere Trainingsloop anhand der momentanen Hyper-Parameter
            rmse = trainingLoop(layer, batchsize, NUM_EPOCHS, hiddendim, NUM_RATINGS, printing = False)
            
            # Berechnungszeit in Sekunden (mit 3 Nachkommastellen)
            end = timer() # Ende Stoppuhr
            timerSeconds = timedelta(seconds=end-start).total_seconds()  # Berechne Zeit
            time = math.ceil(timerSeconds*10)/10 # Runden
            
            # speichere RMSE der Hyper-Parameter und die Parameter selbst
            evalDf = evalDf.append([{'Ebenen' : layer,'Batchgroesse' : batchsize,
                                     'Hiddendims' : hiddendim,'Epochs' : NUM_EPOCHS,
                                     'RMSE' : rmse, 'Zeit (s)' : time}], ignore_index=True)
evalDf.to_csv(r'MovieLens+Feature-100k-10.csv')

#Für die Ausgabe der 'besten' Hyper-Parameter          
bestRMSE = min(evalDf['RMSE']) # ermittle niedrigsten RMSE
zeile = list(evalDf['RMSE']).index(bestRMSE) # ermittle Zeile
print('Niedrigster RMSE-Wert  mit folgenden Hyper-Parametern: \n', evalDf.loc[[zeile]])

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 20.72it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:02<00:00, 10.25it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00,  9.11it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  7.60it/s]
